In [12]:
import ifcopenshell

def open_ifc_file(ifc_file_path):
    print("Opening IFC file...")
    ifc_file = ifcopenshell.open(ifc_file_path)
    print("IFC file opened successfully.")
    return ifc_file

def find_room_by_name(ifc_file, room_name):
    print(f"Searching for the room with name {room_name}...")
    for entity in ifc_file.by_type("IfcSpace"):
        if entity.Name == room_name:
            print(f"Room found: ID={entity.GlobalId}, Name={entity.Name}")
            return entity
    print(f"Room with name {room_name} was not found.")
    return None

def find_adjacent_walls(ifc_file, room):
    adjacent_walls = []
    for rel in ifc_file.by_type("IfcRelSpaceBoundary"):
        if rel.RelatingSpace == room and rel.RelatedBuildingElement:
            if rel.RelatedBuildingElement.is_a("IfcWall"):
                adjacent_walls.append(rel.RelatedBuildingElement)
    return adjacent_walls

def get_wall_geometry(wall):
    try:
        thickness = None
        body_representation = None
        profile_dimensions = None
        extrusion_depth = None
        extrusion_direction = None
        coordinates = []

        representations = wall.Representation.Representations
        for rep in representations:
            if rep.RepresentationType in ["SweptSolid", "Brep", "SurfaceModel", "Tessellation"]:
                body_representation = rep.RepresentationType

            for item in rep.Items:
                if item.is_a("IfcExtrudedAreaSolid"):
                    profile = item.SweptArea
                    if profile.is_a("IfcRectangleProfileDef"):
                        thickness = profile.YDim
                        profile_dimensions = (profile.XDim, profile.YDim)
                    extrusion_depth = item.Depth
                    extrusion_direction = item.ExtrudedDirection.DirectionRatios
                    if item.SweptArea.is_a("IfcArbitraryProfileDefWithVoids"):
                        if item.SweptArea.OuterCurve.is_a("IfcIndexedPolyCurve"):
                            points = item.SweptArea.OuterCurve.Points
                            coordinates = [tuple(point) for point in points.CoordList]

        return profile_dimensions, thickness, extrusion_depth, extrusion_direction, coordinates, body_representation
    except AttributeError as e:
        print(f"Error processing wall {wall.GlobalId}: {e}")
        return None, None, None, None, None, None

def main():
    # Path to the IFC file
    ifc_file_path = "Hus28_test.ifc"
    
    # Open the IFC file
    ifc_file = open_ifc_file(ifc_file_path)
    
    # Find the room with name 923
    room = find_room_by_name(ifc_file, "923")
    if not room:
        return
    
    # Find all adjacent walls for the found room
    adjacent_walls = find_adjacent_walls(ifc_file, room)
    
    # Print the number of adjacent walls found
    print(f"{len(adjacent_walls)} Adjacent Walls found")
    
    # Output the results with profile dimensions, thickness, and representation type
    if adjacent_walls:
        print(f"Details of adjacent walls for room 923 (ID: {room.GlobalId}):")
        for wall in adjacent_walls:
            profile_dimensions, thickness, extrusion_depth, extrusion_direction, coordinates, body_representation = get_wall_geometry(wall)
            print(f"- Wall ID: {wall.GlobalId}, Name: {wall.Name}")
            print(f"  Body Representation: {body_representation}")
            if body_representation == "SweptSolid":
                print(f"  Profile Dimensions: {profile_dimensions}")
                print(f"  Thickness: {thickness} mm")
                print(f"  Extrusion Depth: {extrusion_depth}")
                print(f"  Extrusion Direction: {extrusion_direction}")
                if coordinates:
                    print("  Coordinates of the outer curve:")
                    for point in coordinates:
                        print(f"    {point}")
            elif body_representation == "Tessellation":
                print(f"  Body Representation: Tessellation")
            else:
                print(f"  Body Representation: Different Method")
    else:
        print("No adjacent walls found for room 923.")

if __name__ == "__main__":
    main()


Opening IFC file...
IFC file opened successfully.
Searching for the room with name 923...
Room found: ID=3V7W2RscLEExyQLuCnt1ox, Name=923
6 Adjacent Walls found
Details of adjacent walls for room 923 (ID: 3V7W2RscLEExyQLuCnt1ox):
- Wall ID: 3xDiDpx$nB2hLMfVxR_vR1, Name: Basic Wall:IV31  SCHAKTVÄGG 95:2477100
  Body Representation: SweptSolid
  Profile Dimensions: None
  Thickness: None mm
  Extrusion Depth: 2780.0
  Extrusion Direction: (0.0, 0.0, 1.0)
- Wall ID: 3xDiDpx$nB2hLMfVxR_vR0, Name: Basic Wall:IV01 INNERVÄGG 120:2477101
  Body Representation: SweptSolid
  Profile Dimensions: None
  Thickness: None mm
  Extrusion Depth: 2520.0
  Extrusion Direction: (0.0, 0.0, 1.0)
- Wall ID: 1qLd$aRfnB58C7GyIrSsor, Name: Basic Wall:IV43 ,GIPSVÄGG VID UNDERTAK:3245342
  Body Representation: SweptSolid
  Profile Dimensions: None
  Thickness: None mm
  Extrusion Depth: 45.00000000002442
  Extrusion Direction: (0.0, 0.0, -1.0)
  Coordinates of the outer curve:
    (-1710.625000000002, -2618.75000